## MARKET BASKET ANALYSIS with Apriori

![](https://miro.medium.com/max/2880/1*DHfQvlMVBaJCHpYmj1kmCw.png)

In [ ]:
import numpy as np 
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import mlxtend as ml
print('MXTend Version: %s' % ml.__version__)
print('Pandas Version: %s' % pd.__version__)
print('Numpy Version: %s' % np.__version__)

In [ ]:
df = pd.read_csv('../input/datasets-for-appiori/basket_analysis.csv')
df.head()

In [ ]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
df.head()

**Apriori Analizi Kuralları**
* Veri seti tabular veya transactional yapıda olmalıdır
* Veriler kategorik olmalıdır.
* Verideki değişkenlerin yönleri in,out veya both olarak tanımlanmalıdır.

* **Not:** Veri setini içerik aktardıktan sonra nested list (iç içe liste) tipinde ise tabular bir yapıya çevirmemiz gerekiyor. Bunun için mlxtend modülü içerisinde yer alan preprocessing sınıfı içerisinde TransactionEncoder fonksiyonunu kullanabilirsiniz. Bizim bu veri setinde bu işleme ihtiyacımız yok.

<code>from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(df).transform(df)
df = pd.DataFrame(te_ary, columns=te.columns_)</code>

*TransactionEncoder hakkında daha fazla bilgi için: http://rasbt.github.io/mlxtend/user_guide/preprocessing/TransactionEncoder/*

In [ ]:
df.info()

**Model Oluşturma**<br>

<code>from mlxtend.frequent_patterns import apriori</code>

*Apriori hakkında daha fazla bilgi için : http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/*

In [ ]:
apriori(df, min_support=0.15)[1:25]

Tablodaki itemset kolonu içerisinde yazan rakamları ürünleri (0-15) ifade etmektedir. 0 no’lu ürün Apple, 1 no’lu ürün Bread olarak 14 no’lu ürün ise Yogurt ifade etmektedir.
<br><br>
 0   Apple         999 non-null    bool<br> <br>
 1   Bread         999 non-null    bool<br>  
 2   Butter        999 non-null    bool<br>  
...

In [ ]:
print("Kural Sayısı:", len(apriori(df, min_support=0.15)))

Şimdi de apriori algoritması içerisinde use_colnames=True parametresini kullanarak items(ürünler) numaralarından item(ürün) isimlerine geçiş yapıyoruz.

In [ ]:
apriori(df, min_support=0.15, use_colnames=True)[1:25]

Yukarıdaki tabloda tekli, ikili ve üçlü itemset’lerin oluştuğu görülmektedir. min_support değeri (0.15) değerini set edip kuralları rule setleri oluşturduktan sonra ilgilendiğimiz metriğe göre (confidence, lift, conviction ve vd.) Association Rules tablosunu oluşturuyoruz. Burada metric olarak Confidence ve değerini 0.3 (%30) seçtik.

In [ ]:
frequent_itemsets = apriori(df, min_support=0.15, use_colnames=True)
rules1 = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.30)

**Kural Sayıları**

<code>from mlxtend.frequent_patterns import association_rules</code>

*Association_rules hakkında daha fazla bilgi için: http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/*

In [ ]:
print("Oluşan Kural Sayısı:", len(rules1))

**Confidence metriğine göre (Z-A) 10 Kural:**

In [ ]:
rules1 = rules1.sort_values(['confidence'], ascending=False)

rules1[1:11]

**Yorum 1:** ID bilgisi 67 olan satırı inceleyecek olursak;
* Ica Cream ve Butter item’larının birlikte görülme olasılığı (support) %21 (0.207) olduğunu,
* Ice Cream item’ının satın alan kişilerin (confidence)  %50’inin (0.504878) olasılıkla  Butter item’ınıda satın aldığını,
* Ice Cream item’ının yer aldığı alışveriş sepetlerin de Butter item’ının satışı (lift) 1,20 kat arttığı,
* Ice Cream ve Butter item’larının birlikte satın alınmalarının ile birbirlerinden bağımsız olarak satın alınmalarından ne kadar fazla (leverage) 0.03 olduğunu,
* Ice Cream ve Butter item’larının birbirleri ile ilişkili (conviction) 1.17 değeri ile olduğunu  söyleyebiliriz.

Şimdi de antecedents ve consequents kısımlarında bulunan itemsleri sayılarını toplayalım ve ilk 5 satırı görelim:

In [ ]:
rules1["antecedent_len"] = rules1["antecedents"].apply(lambda x: len(x))
rules1["consequents_len"] = rules1["consequents"].apply(lambda x: len(x))
rules1[1:6]

Yukarıdaki confidence metriği için yaptıklarımızı diğer metrikler içinde yapabiliriz. Örnek olması adına lift metriği için:

In [ ]:
rules2 = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules2 = rules2.sort_values(['lift'], ascending=False)
rules2[1:6]

In [ ]:
rules2["antecedent_len"] = rules2["antecedents"].apply(lambda x: len(x))
rules2["consequents_len"] = rules2["consequents"].apply(lambda x: len(x))
rules2[1:6]

**Oluşan Kural Setleri için Filtreleme**

Filtre 1: Antecedent item uzunluğu 1 olan ve Confidence değeri 0.20’ye büyük eşit olan ve Lift değeri 1‘den büyük olan ilk 10 kayıtı görelim.

In [ ]:
rules1[(rules1['antecedent_len'] >= 1) &
       (rules1['confidence'] >= 0.20) &
       (rules1['lift'] > 1) ].sort_values(['confidence'], ascending=False)[1:10]

Filtre 2: Benzer şekilde Antecedents item adı Bread olan ve Confidence metriğine göre [Z-A] sıralanmış ilk 10 kayıt:

In [ ]:
rules1[rules1['antecedents'] == {'Bread'}].sort_values(['confidence'], ascending=False)[1:10]

Verilen parametre değerlerine göre oluşan Birliktelik Kuralları Analizine ait oluşan kuralları <code>.json</code> olarak olarak export ediyoruz.

In [ ]:
rules1.to_json('./rules1.json')
rules2.to_json('./rules2.json')

http://rasbt.github.io/mlxtend/<br>
https://github.com/rasbt/mlxtend<br>
https://pandas.pydata.org/<br>
https://www.veribilimiokulu.com/python-ile-birliktelik-kurallari-analizi-association-rules-analysis-with-python/<br>